In [1]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq

In [2]:
client, llm = init_groq()

In [3]:
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"

In [4]:
my_tools = []


my_tools.append(
    
    GoogleDocWriteTool()
)

my_tools.append(
    
    GoogleDriveUploadTool()
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
)
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

my_tools.append(
    ImprovedSearchTool(credentials_path)
)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64661%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=BptFomIUCyDtHmucuuNDktcYe5LXxo&access_type=offline
Authenticating with Google Drive API...
Running local server for authentication...
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64668%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=iRXi9cjnAH5eGU1Ac3sf8p3kJlAmM5&access_type=offline
Authentication successful.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.

In [5]:
from langchain.agents import load_tools
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]
my_tools.append(humantool)

In [6]:
# Chat history tracking
chat_history = []

In [9]:
chat_history

[{'input': '\nDo you remember what we just talked about? Answer without any tools. \nNow, you can use tools. Please upload document to google drive. \n',
  'response': {'input': '\nDo you remember what we just talked about? Answer without any tools. \nNow, you can use tools. Please upload document to google drive. \n',
   'chat_history': [...],
   'tools': "GoogleDocWriteTool: Writes any amount of content to a Google Doc with professional formatting. Inputs are the text to write, and whether to append or not, as well as document name\nGoogleDriveUploadTool: Uploads a PDF to Google Drive and sets permissions for a specific user. Please set the 'rename' parameter to None if the user does not want to rename the file before uploading to Google Drive. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION\nGoogleSheetsUpdateTool: Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION\nGmailSendPdfTool: Sends an email

In [14]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

You need credentials of the user. Here they are:
{name}\n
{email}\n
{recipient_email}\n
{phone}\n
-------
HERE IS PATH to documents:
{thepath}
-------
Here are the tools you can use. DO NOT TRY TO USE ANYTHING OTHER THAN THESE:
{tools}
-------
HERE IS GUIDANCE ON EXACTLY WHAT TO PASS IN TO THE TOOLS:
GoogleDocWriteTool: INPUT EVERYTHING CORRECTLY EVEN IF ITS LONG!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! NO DICTIONARIES, JUST EVERYTHING PASSED AS IT ISSS!!!!!--> INPUT_TEXT: generate it yourself | APPEND: decide whether or not you want to append to document (must be in lowercase)
MoveFileTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FILE_NAME please deduce from the response and then pass it into tool: {input} ; for rest of the parameters, there will be instructions given over time for you.
GoogleSheetsUpdateTool: Input ONLY ONLY these into the tool PROPERLY! --> {name}, {phone}
CreateFolderTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER NAME: the folder name as specified in response --> {input}; pass in 'root' as the value for parent_folder_id parameter if user indicates 'google drive' or 'my drive' ||| PLEASE OUTPUT THE EXACT NAME OF THE FOLDER THAT WAS CREATED
FolderMovementTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> FOLDER_NAME please deduce from the response and then pass it into tool: {input} ; NEW_PARENT_FOLDER_NAME also deduce from response and PASS IT IN {input}. SPECIAL CASE - PASS IN 'root' if user says move into google drive or my drive: {input}. The IDs you will find out later. for now they are optional.
FileOrganizerTool: INPUT DIRECTLY THE FOLLOWING INPUTS --> PARENT_FOLDER_NAME please deduce from the response and then pass it into tool - PASS IN 'root' if user says move into google drive or my drive: {input}
GoogleDriveUploadTool: INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {email}
GmailSendPdfTool: IF THERE IS A FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}
GmailSendPdfTool: IF THERE IS NO FILE TO SEND, INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {email}, RECIPIENT EMAIL: {recipient_email}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing
ImprovedSearchTool: DEDUCE THE FILE NAME OR FOLDER NAME from response --> {input}, and pass it in to this tool; AND if you know the ID you can pass it in as well.

If what the user says is one of these you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [insert name(s) of whatever tool(s) to be used]". 
so that user can confirm if you got it correctly. 
When writing to a file, just start ONLY ONLY ONLY THE BEGINNING OF THE ENTIRE CONTENT with '@' symbol when generating file content. There should only be ONNNEEEEE '@' symbol in response and that is at the beginning!!!!!
WHEN WRITING TO FILE--> If user indicates that they want to add on to existing content, \
you must say 'yes append' before the aforementioned '@' symbol. OTHERWISE YOU SAY 'no append' before the aforementioned '@' symbol

IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.


IMPORTANT: 1. THIS IS FOR ANYTHING INVOLVING FILES OR FOLDERS:
    - MAKE SURE THAT ID'S ARE 100% CORRECT! IF ID'S ARE WRONG, LIFE WILL END AS WE KNOW IT!
   - First, use the ImprovedSearchTool to find the file ID for the file name provided.
   - Then, use the ImprovedSearchTool to find the folder ID for the folder name provided.
   - Once you have both IDs, use the MoveFileTool to move the file to the specified folder.


If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.


---------
Chat History: {chat_history}
User Input: {input}
"""


llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

from langchain.agents.structured_chat.prompt import FORMAT_INSTRUCTIONS
prompt_template = ChatPromptTemplate.from_template(
    template="If you say or do anything other than what is in here, life will end as we know it. \n\n" + FORMAT_INSTRUCTIONS + "\n\n" + template
)

agent = initialize_agent(
    tools=my_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    prompt_template=prompt_template,
    verbose=True,
    max_iterations=1000
)


print('PROMPT TEMPLATE\n\n',str(prompt_template))


# User credentials
credentials = {
    "name": "Sriram Nallani",
    "email": "sriramnallani35@gmail.com",
    "recemail": "thegreatchrisjohn@gmail.com",
    "phone": "4973993399"
}

# Google TTS client initialization
def synthesize_speech(text, output_file):
    client = texttospeech.TextToSpeechClient()
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')

# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    tools_str = '\n'.join([f"{tool.name}: {tool.description}" for tool in my_tools])
    response = agent.invoke({
        "input": input_text,
        "chat_history": chat_history,
        "tools": tools_str,
        "name": credentials['name'],
        "email": credentials['email'],
        "recipient_email": credentials['recemail'],
        "phone":credentials['phone'],
        "thepath": "c:\\DEV\\WEBDEVFOLDER\\REALESTATEAI\\DOCUMENTS\\MYDOCUMENT.PDF"
    })
    chat_history.append({"input": input_text, "response": response})
    
    # Use Google TTS to convert the response to speech
    #tts_output_file = "output.mp3"
    #synthesize_speech(response, tts_output_file)
    #print(f'Response audio saved to {tts_output_file}')
    
    return response

# Can you please write a 5 paragraph essay about toilets and their relation to artificial intelligence, 
# and then move the document to a folder called FCPS google. 
# Then create a folder named 'dummy folder 2' and then move that to folder called 'LM'. 

thepath= "c:\\DEV\\WEBDEVFOLDER\\REALESTATEAI\\DOCUMENTS\\MYDOCUMENT.PDF"
user_input = f""" Please upload document to google drive. Info: {credentials} and {thepath}. move it into documents_mydrive folder"""
result = run_agent(user_input)
print(result['output'])


PROMPT TEMPLATE

 input_variables=['chat_history', 'email', 'input', 'name', 'phone', 'recipient_email', 'thepath', 'tool_names', 'tools'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history', 'email', 'input', 'name', 'phone', 'recipient_email', 'thepath', 'tool_names', 'tools'], template='If you say or do anything other than what is in here, life will end as we know it. \n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{{{\n  "action": "Fi

TypeError: MoveFileTool._run() got an unexpected keyword argument 'file_path'